In [16]:
using Revise

using BilevelTrajOpt
using LinearAlgebra
using ForwardDiff

In [4]:
a = rand(4)
A = Matrix(Diagonal(a))

4×4 Array{Float64,2}:
 0.9575  0.0        0.0      0.0      
 0.0     0.0128424  0.0      0.0      
 0.0     0.0        0.19172  0.0      
 0.0     0.0        0.0      0.0898911

In [55]:
# test the solution
function f(x)
    return sum(x)
end

function h(x)
    return [6. .- x[1], x[1]*x[2], x[1]*.5 + x[3] - 4.]
end

function g(x)
    return vcat(-x[2:3],5. - x[4],10. - x[4],17. - x[4],5. - x[1]*x[4])
end

x0 = zeros(4)
λ0 = zeros(length(h(x0)))
μ0 = zeros(length(g(x0)))
c0 = 1.

x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g,c0=c0)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g)

println(x_sol)
println(x_sol_ip)

Solve_Succeeded
[6.0, 1.50878e-12, 1.0, 17.0]
[6.0, 2.16408e-24, 1.0, 17.0]


In [56]:
# test the gradient
function solve_prob(z)
    function f(x)
        return sum(x)
    end

    function h(x)
        return [z[1] - x[1], x[1]*x[2], x[1]*.5 + x[3] - z[2]]
    end

    function g(x)
        return vcat(-x[2:3], z[3] - x[4], 10. - x[4], z[4] - x[4], 5. - x[1]*x[4])
    end

    x0 = zeros(4)
    λ0 = zeros(length(h(x0)))
    μ0 = zeros(length(g(x0)))

    # x_sol_ip = ip_solve(x0,f,h,g,length(h(x0)),length(g(x0)))
    # println(x_sol_ip)
    
    x_sol, λ_sol, μ_sol, c_sol = auglag_solve(x0,λ0,μ0,f,h,g)
    # println(x_sol)

    x_sol
end

solve_prob (generic function with 1 method)

In [103]:
# z0 = [2.,-1.,0.,1.]
z0 = [6.,4.,5.,17.]

sol = solve_prob(z0)

# autodiff 
J_auto = ForwardDiff.jacobian(solve_prob,z0)

# numerical
ϵ = sqrt(eps(1.))
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - sol) ./ ϵ
end

println("----")
display(sol)
display(J_auto)
display(J_num)
display(maximum(abs.(J_auto - J_num)))

4-element Array{Float64,1}:
  5.400794044375547   
  0.006073391694890981
  0.935586826537582   
 14.329164068888042   

4×4 Array{Float64,2}:
  0.897474     0.00945254   -3.73698e-13   2.61753e-11
  7.87403e-5  -0.000279967   1.46313e-14  -3.51195e-13
 -0.439284     0.894099      1.93308e-13  -1.31759e-11
 -0.200563    -0.0428933    -0.00396176    0.902163   

4×4 Array{Float64,2}:
  0.897474     0.00945437    2.32458e-6  2.5034e-6 
  7.96301e-5  -0.000279587   6.77072e-7  1.08092e-6
 -0.439283     0.894098      2.90573e-6  3.02494e-6
 -0.200566    -0.0428977    -0.0039643   0.902161  

4.449259473904288e-6

----


In [97]:
display(maximum(abs.(J_auto - J_num)))

0.002547256759153016